<a href="https://colab.research.google.com/github/GenericP3rson/WikiGraph/blob/main/WikiGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WikiGraph

## Schema Creation

In [ ]:
!pip install pyTigerGraph[gds]

import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host = "https://bleve.i.tgcloud.io/", graphname = "YourGraph")
conn.apiToken = conn.getToken(conn.createSecret())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 5.2 MB/s 
     |████████████████████████████████| 246 kB 14.8 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=c6b4c03daa448ffa435718ea4b5f18cc9ab2423e05bcff5afb70dd8d53560053
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
conn.gsql('''
CREATE VERTEX Doc(PRIMARY_ID id STRING, content STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"
CREATE VERTEX Entity(PRIMARY_ID entity STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"

CREATE DIRECTED EDGE LINKS_TO(FROM Doc, TO DOC, weight DOUBLE)
CREATE UNDIRECTED EDGE DOC_ENTITY(FROM Doc, TO ENTITY, weight DOUBLE)

CREATE GRAPH WikiGraph(Doc, Entity, LINKS_TO, DOC_ENTITY)
''')

## Load Data

In [ ]:
import requests
from bs4 import BeautifulSoup
import spacy

# Text cleaning, entity extraction, sentiment analysis, summarisation
# Graph structures

nlp = spacy.load("en_core_web_sm")

seen = []

def scrape(input_link):

  response = requests.get(
    url = input_link,
  )
  soup = BeautifulSoup(response.content, 'html.parser')

  first_paragraph = soup.find('p')

  while first_paragraph.get_text().strip() == "":
    first_paragraph = first_paragraph.find_next_sibling('p')

  conn.upsertVertex("Doc", input_link, attributes={"id": input_link, "content": first_paragraph.get_text()})

  ent = nlp(first_paragraph.get_text())
  
  for e in ent:
    conn.upsertVertex("Entity", str(e), attributes={"entity": str(e)})
    conn.upsertEdge("Doc", input_link, "DOC_ENTITY", "Entity", str(e))

  seen.append(input_link)

  links = [f"https://en.wikipedia.org{link['href']}" for link in first_paragraph.find_all("a") if link.has_attr("href") and link["href"][:6] == "/wiki/"]
  for link in links:
    conn.upsertEdge("Doc", input_link, "LINKS_TO", "Doc", link)
    if link not in seen:
      scrape(link)

33scrape("https://en.wikipedia.org/wiki/Python_(programming_language)")

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


## Queries

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")

search = "graph database"
terms = nlp(search)

conn.upsertVertex("Doc", "search_doc", attributes = {"id": "search_doc", "content": search})
for i in terms:
  conn.upsertEdge("Doc", "search_doc", "DOC_ENTITY", "Entity", str(i))

conn.runInstalledQuery("tg_jaccard_nbor_ss", params = {"source": ("search_doc", "Doc"), "e_type": "DOC_ENTITY", "rev_e_type": "DOC_ENTITY"})

conn.deleteVertex("Doc", "search_doc")

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


[{'Others': [{'attributes': {'Others.@sum_similarity': 0.025},
    'v_id': 'https://en.wikipedia.org/wiki/Graph_database',
    'v_type': 'Doc'}]}]